In [2]:
import torch
if torch.cuda.is_available():
    import torch.cuda as t
else:
    import torch as t
    
from   torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
import json, os
from pathlib import Path

In [3]:
class SimpleCNN(nn.Module):
    def __init__(self, n_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 4, 3)
        self.dropout = nn.Dropout()    
        self.fc1 = nn.Linear(4*26*26,100)
        self.fc2 = nn.Linear(100, n_classes)
            
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = x.view(-1,4*26*26)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

In [4]:
config = json.load(open(os.path.expanduser("~/.thesis.conf")))
root = Path(config['datasets']) / Path("MNIST/") 
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
mnist_set = torchvision.datasets.MNIST(str(root),train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(mnist_set, batch_size=25,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.MNIST(str(root), train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

dataiter = iter(trainloader)
images, label = dataiter.next()

In [5]:
type(mnist_set)

torchvision.datasets.mnist.MNIST

In [23]:
net = SimpleCNN(10).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
loss_
for epoch in range(3):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = Variable(inputs).cuda(), Variable(labels).cuda()
        
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.data[0]
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 2.302
[1,   400] loss: 2.298
[1,   600] loss: 2.294
[1,   800] loss: 2.281
[1,  1000] loss: 2.228
[1,  1200] loss: 2.060
[1,  1400] loss: 1.892
[1,  1600] loss: 1.805
[1,  1800] loss: 1.767
[1,  2000] loss: 1.726
[1,  2200] loss: 1.723
[1,  2400] loss: 1.705
[2,   200] loss: 1.697
[2,   400] loss: 1.682
[2,   600] loss: 1.677
[2,   800] loss: 1.681
[2,  1000] loss: 1.673
[2,  1200] loss: 1.679
[2,  1400] loss: 1.670
[2,  1600] loss: 1.667
[2,  1800] loss: 1.669
[2,  2000] loss: 1.671
[2,  2200] loss: 1.666
[2,  2400] loss: 1.667
[3,   200] loss: 1.664
[3,   400] loss: 1.669
[3,   600] loss: 1.654
[3,   800] loss: 1.653
[3,  1000] loss: 1.654
[3,  1200] loss: 1.657
[3,  1400] loss: 1.652
[3,  1600] loss: 1.653
[3,  1800] loss: 1.653
[3,  2000] loss: 1.646
[3,  2200] loss: 1.658
[3,  2400] loss: 1.655
Finished Training


In [22]:
correct = 0
total = 0
for data in testloader:
    images, labels = data
    imgs = Variable(images).cuda()
    outputs = net(imgs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 81 %
